In [1]:
# setwd
import os
os.chdir('C:/Users/ASUS/Desktop/Daming/VSC/stock_price_prediction_with_realtime_evaluation')

# Scrap Dataset

In [23]:
import time
from selenium.webdriver.common.keys import Keys#untuk tekan tekan
from selenium.webdriver.common.by import By#untuk nyari element
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import selenium
import bs4
from datetime import datetime,timedelta
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [5]:
def ambil_data_perusahaan(perusahaan):
    # jadikan variable perusahaan global
    list_hari=[]
    list_harga=[]
    
    try :
        global pt
        pt=perusahaan
        
        #buat date
        start_date = '2020-01-01'
        now = datetime.now()
        one_day_before = now - timedelta(days=1)
        end_date = one_day_before.strftime("%Y-%m-%d")
        dates = pd.date_range(start=start_date, end="2020-02-01")

        # Memformat tanggal ke format DD-MM-YYYY
        formatted_dates = [date.strftime('%Y-%m-%d') for date in dates]
        
        
        #siapkan driver
        #headless
        #cara buat driver headless
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("window-size=1920,1080")
        chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36')
        
        driver = selenium.webdriver.Chrome(options=chrome_options)
        url="https://www.idx.co.id/id/data-pasar/ringkasan-perdagangan/ringkasan-saham/"
        driver.get(url)
        time.sleep(2)

        #tulis nama perusahaan
        nama_perusahaan=driver.find_element(By.CSS_SELECTOR,"#FilterSearch")
        nama_perusahaan.send_keys(perusahaan)

        #buat tabel data
        for i in formatted_dates:
            tanggal=driver.find_element(By.CSS_SELECTOR,"#app > div.sticky-footer-container-item.--pushed > main > div > div.mb-24.filter > div.filter-items > div.full-responsive.filter-center > div.mx-datepicker > div > input")
            tanggal.click()
            tanggal.clear()
            tanggal.send_keys(i)
            nama_perusahaan.click()
            try:
                WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#app > div.sticky-footer-container-item.--pushed > main > div > div.vgt-boxed.mb-24.vgt-wrap > div")))
                
                soup=bs4.BeautifulSoup(driver.page_source,"html.parser")
                postingan=soup.find_all('td',class_="vgt-right-align")
                if postingan==[]:
                    harga="unk"
                else:
                    harga=postingan[2].text.strip()
            except:
                harga="unk"
                
            list_hari.append(i)
            list_harga.append(harga)

        #hasilkan dataframe
        df=pd.DataFrame({"tanggal":list_hari,"harga":list_harga})
        df.to_csv(f"data/raw/{perusahaan}.csv",index=False)
        driver.quit()
        return df
    except:
        df=pd.DataFrame({"tanggal":list_hari,"harga":list_harga})
        df.to_csv(f"data/raw/{perusahaan}.csv",index=False)
        driver.quit()
        return df


## Pilih perusahaan

In [3]:
data_perusahaan=ambil_data_perusahaan("AALI")
data_perusahaan

NameError: name 'ambil_data_perusahaan' is not defined

# Preprocessing

In [29]:
df=pd.read_csv("data/raw/data_mandiri.csv")

# misal ada data baru yang masuk
def data_processing(df1):
    #buat df dengan tanggal dari 2020-01-01 ke 2024-02-23 dan kolom harga saham berisi harga "unk"
    now = datetime.now()
    one_day_before = now - timedelta(days=1)
    date_rng=pd.date_range(start='1/1/2020', end=one_day_before, freq='B')

    #buat df dengan tanggal dari 2020-01-01 ke 2024-02-23 dan kolom harga saham berisi harga "unk"
    df2=pd.DataFrame(date_rng, columns=['tanggal'])
    df2['harga_saham']='unk'

    #ubah tipe data tanggal menjadi datetime
    df1['tanggal']=pd.to_datetime(df1['tanggal'])
    df2['tanggal']=pd.to_datetime(df2['tanggal'])
    df=pd.merge(df1, df2, on='tanggal', how='outer')

    #mengisi missing value dengan "unk"
    df=df[["tanggal","harga"]].fillna("unk")
    df['harga']=df['harga'].replace('unk', np.nan)
    
    #ubah tipe data harga menjadi float
    df["harga"]=df["harga"].astype(float)
    # mengisi missing value dengan nilai interpolasi
    df['harga']=df['harga'].interpolate()

    #isi lagi nan dengan nilai ffill
    df['harga']=df['harga'].fillna(method='ffill')

    #isi lagi nan dengan nilai bfill
    df['harga']=df['harga'].fillna(method='bfill')

    # df.to_csv(f"data/processed/{pt}.csv",index=False)  
    return df

data_processing(df)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_4768\512382440.py:29: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['harga']=df['harga'].fillna(method='ffill')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_4768\512382440.py:32: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['harga']=df['harga'].fillna(method='bfill')


,tanggal,harga
0,2020-01-01,7.750
1,2020-01-02,7.750
2,2020-01-03,7.725
3,2020-01-06,7.600
4,2020-01-07,7.600
...,...,...
1083,2024-02-26,7.050
1084,2024-02-27,7.050
1085,2024-02-28,7.050
1086,2024-02-29,7.050


# EDA

In [ ]:
data_preprocessed=pd.read_csv(f"data/processed/{pt}.csv")
data_preprocessed

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

#plot data
plotdf=data_preprocessed.copy()
fig=px.line(plotdf,x='tanggal',y='harga',labels={'x':'Tanggal','y':'Harga Saham'},title=f"Harga Saham {pt}")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()


# Modelling

In [11]:

#untuk harian
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

def buat_model(df):
    # normalisasi data
    data_model=df.copy()
    scaler=MinMaxScaler(feature_range=(0,1))
    data_model["norm"]=scaler.fit_transform(data_model[["harga"]])

    # ambil kolom norm
    data_norm=data_model[["norm"]].values.reshape(-1,1)

    # ubah ke tensor untuk lstm
    time_step = 15
    X_train, y_train = create_dataset(data_norm, time_step)

    #reshape input to be [samples, time steps, features] which is required for LSTM
    model=Sequential()
    model.add(LSTM(32,return_sequences=True,input_shape=(time_step,1)))
    model.add(LSTM(32,return_sequences=True))
    model.add(LSTM(32))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',optimizer='adam')
    model.fit(X_train,y_train,epochs=200,batch_size=5,verbose=1)
    
    ### Lets Do the prediction and check performance metrics
    train_predict=model.predict(X_train)
    # Transform back to original form
    train_predict = scaler.inverse_transform(train_predict)
    original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
    score=r2_score(original_ytrain, train_predict)
    print(f"R2 Score: {score}")


In [31]:
#untuk bulanan
data_model=pd.read_csv("data/processed/data_model.csv")
def buat_model_bulanan(df):
    # normalisasi data
    data_model=df.copy()
    data_model["harga"]=data_model["harga"].astype(float)
    data_model["tanggal"]=data_model["tanggal"].astype(str)
    data_model["tanggal"]=[i[:7] for i in data_model["tanggal"]]
    data_model=data_model.groupby("tanggal")["harga"].mean().reset_index()
    
    scaler=MinMaxScaler(feature_range=(0,1))
    data_model["norm"]=scaler.fit_transform(data_model[["harga"]])

    # ambil kolom norm
    data_norm=data_model[["norm"]].values.reshape(-1,1)

    # ubah ke tensor untuk lstm
    time_step = 7
    X_train, y_train = create_dataset(data_norm, time_step)

    #reshape input to be [samples, time steps, features] which is required for LSTM
    model=Sequential()
    model.add(LSTM(32,return_sequences=True,input_shape=(time_step,1)))
    model.add(LSTM(32,return_sequences=True))
    model.add(LSTM(32))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',optimizer='adam')
    model.fit(X_train,y_train,epochs=200,batch_size=5,verbose=1)
    
    ### Lets Do the prediction and check performance metrics
    train_predict=model.predict(X_train)
    # Transform back to original form
    train_predict = scaler.inverse_transform(train_predict)
    original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
    score=r2_score(original_ytrain, train_predict)
    print(f"R2 Score: {score}")

In [32]:
buat_model_bulanan(data_model)



Epoch 1/200

9/9 [==============================] - 3s 6ms/step - loss: 0.2603
Epoch 2/200
9/9 [==============================] - 0s 5ms/step - loss: 0.1494
Epoch 3/200
9/9 [==============================] - 0s 6ms/step - loss: 0.0660
Epoch 4/200
9/9 [==============================] - 0s 6ms/step - loss: 0.0602
Epoch 5/200
9/9 [==============================] - 0s 6ms/step - loss: 0.0570
Epoch 6/200
9/9 [==============================] - 0s 6ms/step - loss: 0.0543
Epoch 7/200
9/9 [==============================] - 0s 6ms/step - loss: 0.0566
Epoch 8/200
9/9 [==============================] - 0s 6ms/step - loss: 0.0553
Epoch 9/200
9/9 [==============================] - 0s 6ms/step - loss: 0.0571
Epoch 10/200
9/9 [==============================] - 0s 6ms/step - loss: 0.0550
Epoch 11/200
9/9 [==============================] - 0s 5ms/step - loss: 0.0544
Epoch 12/200
9/9 [==============================] - 0s 5ms/step - loss: 0.0554
Epoch 13/200
9/9 [==============================] - 0s 5ms

# Comparison predicted vs original

In [ ]:
# shift train predictions for plotting
def plot_prediction(data_norm,train_predict):
    look_back=time_step
    trainPredictPlot = np.empty_like(data_norm)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
    print("Train predicted data: ", trainPredictPlot.shape)

    names = cycle(['Original harga price','Train predicted harga price','Test predicted harga price'])


    plotdf = pd.DataFrame({'date': data['tanggal'],
                        'original_harga': data['harga'],
                        'train_predicted_harga': trainPredictPlot.reshape(1,-1)[0].tolist(),
                        })

    fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_harga'],plotdf['train_predicted_harga'],
                                            ],
                labels={'value':'Stock price','date': 'Date'})
    fig.update_layout(title_text='Comparision between original harga price vs predicted harga price',
                    plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
    fig.for_each_trace(lambda t:  t.update(name = next(names)))

    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)
    fig.show()

# Testing the model

In [ ]:
def prediksi_harga_saham(nhari):
        
    df=data_preprocessed.copy()
    scaler=MinMaxScaler(feature_range=(0,1))
    df["norm"]=scaler.fit_transform(df[["harga"]])
    
    x_input=df[["norm"]][len(df)-time_step:].values.reshape(1,-1)
    temp_input=list(x_input)
    temp_input=temp_input[0].tolist()
    
    #prediksi sebanyak data baru
    from numpy import array

    lst_output=[]
    n_steps=time_step
    i=0
    pred_days = nhari
    while(i<pred_days):
        
        if(len(temp_input)>time_step):
            
            x_input=np.array(temp_input[1:])
            #print("{} day input {}".format(i,x_input))
            x_input = x_input.reshape(1,-1)
            x_input = x_input.reshape((1, n_steps, 1))
            
            yhat = model.predict(x_input, verbose=0)
            #print("{} day output {}".format(i,yhat))
            temp_input.extend(yhat[0].tolist())
            temp_input=temp_input[1:]
            #print(temp_input)
        
            lst_output.extend(yhat.tolist())
            i=i+1
            
        else:
            
            x_input = x_input.reshape((1, n_steps,1))
            yhat = model.predict(x_input, verbose=0)
            temp_input.extend(yhat[0].tolist())
            
            lst_output.extend(yhat.tolist())
            i=i+1
            
    # Konversi prediksi menjadi format yang sesuai dan menambahkannya ke DataFrame
    lst_output = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]
    
    #masukan hasil ke csv
    now = datetime.now()
    one_day_before = now
    one_day_until=  now + timedelta(days=nhari-1)
    date_rng=pd.date_range(start=one_day_before, end=one_day_until, freq='B')

    data_baru=pd.DataFrame(date_rng, columns=['tanggal'])
    data_baru["tanggal"]=pd.to_datetime(data_baru["tanggal"])
    data_baru['prediksi']=lst_output
    
    
    print("Output of predicted next days: ", len(lst_output))
    return data_baru
        

## Prediksi harga saham selama n hari ke depan

In [ ]:
def plot_prediksi():
    # buat plotly untuk data baru
    names = cycle(['Predicted harga price'])
    plotdf = pd.DataFrame({'date': data_baru['tanggal'],            
                        'predicted_harga': data_baru['prediksi']})

    fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_harga'],plotdf['predicted_harga']],
                    labels={'value':'Stock price','date': 'Date'})
    fig.update_layout(title_text='Comparision between original harga price vs predicted harga price',
                    plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
    fig.for_each_trace(lambda t:  t.update(name = next(names)))

    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)
    fig.show()

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from itertools import cycle

# Asumsikan 'data' dan 'data_baru' sudah didefinisikan sebelumnya
# 'trainPredictPlot' juga harus sudah diolah sebelumnya

# Siapkan DataFrame untuk plotting
plotdf = pd.DataFrame({
    'date': data_preprocessed['tanggal'],  # pastikan 'data' sudah didefinisikan
    'original_harga': data_preprocessed['harga'],  # pastikan 'data' sudah didefinisikan
    'train_predicted_harga': trainPredictPlot.reshape(-1).tolist(),  # sesuaikan dengan data Anda
})

# Membuat plot dengan Plotly
fig = px.line(plotdf, x='date', y=['original_harga', 'train_predicted_harga'], labels={'value': 'Harga', 'variable': 'Kategori'})

# Menentukan nama untuk setiap trace dengan menggunakan cycle
names = cycle(['Harga Original', 'Harga Prediksi Pelatihan'])

# Update nama untuk setiap trace
fig.for_each_trace(lambda t: t.update(name=next(names)))

# Tambahkan data prediksi baru dengan garis merah
fig.add_trace(go.Scatter(
    x=data_baru['tanggal'], y=data_baru['prediksi'], mode='lines',
    name='Prediksi Harga Baru (Data baru)', line=dict(color='green'),
    hovertemplate='Kategori :Prediksi Harga Baru (Data baru)<br>Tanggal: %{x}<br>Harga Prediksi: %{y}<extra></extra>'
))


# Update layout
fig.update_layout(
    title='Perbandingan Harga Original vs Prediksi',
    plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Harga'
)

# Update tampilan sumbu
fig.update_xaxes(showgrid=False, title_text='Tanggal')
fig.update_yaxes(showgrid=False, title_text='Harga')

# Tampilkan plot
fig.show()


# Data baru yang masuk

In [ ]:
# buat plotly untuk data baru
names = cycle(['Original harga price','Predicted harga price'])
plotdf = pd.DataFrame({'date': data_baru['tanggal'],                   
                      'original_harga': data_baru['harga'],
                      'predicted_harga': data_baru['prediksi']})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_harga'],plotdf['predicted_harga']],
                labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original harga price vs predicted harga price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

## Gabungkan plotly

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from itertools import cycle

# Asumsikan 'data' dan 'data_baru' sudah didefinisikan sebelumnya
# 'trainPredictPlot' juga harus sudah diolah sebelumnya

# Siapkan DataFrame untuk plotting
plotdf = pd.DataFrame({
    'date': data['tanggal'],  # pastikan 'data' sudah didefinisikan
    'original_harga': data['harga'],  # pastikan 'data' sudah didefinisikan
    'train_predicted_harga': trainPredictPlot.reshape(-1).tolist(),  # sesuaikan dengan data Anda
})

# Membuat plot dengan Plotly
fig = px.line(plotdf, x='date', y=['original_harga', 'train_predicted_harga'], labels={'value': 'Harga', 'variable': 'Kategori'})

# Menentukan nama untuk setiap trace dengan menggunakan cycle
names = cycle(['Harga Original', 'Harga Prediksi Pelatihan'])

# Update nama untuk setiap trace
fig.for_each_trace(lambda t: t.update(name=next(names)))

# Tambahkan data prediksi baru dengan garis merah
fig.add_trace(go.Scatter(
    x=data_baru['tanggal'], y=data_baru['prediksi'], mode='lines',
    name='Prediksi Harga Baru (Data baru)', line=dict(color='green'),
    hovertemplate='Kategori :Prediksi Harga Baru (Data baru)<br>Tanggal: %{x}<br>Harga Prediksi: %{y}<extra></extra>'
))

# Tambahkan titik data original dengan marker hitam
fig.add_trace(go.Scatter(
    x=data_baru['tanggal'], y=data_baru['harga'], mode='markers',
    name='Harga Original (Data Baru)', marker=dict(color='black'),
    hovertemplate='Kategori :Harga Original (Data Baru)<br>Tanggal: %{x}<br>Harga Original: %{y}<extra></extra>'
))

# Update layout
fig.update_layout(
    title='Perbandingan Harga Original vs Prediksi',
    plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Harga'
)

# Update tampilan sumbu
fig.update_xaxes(showgrid=False, title_text='Tanggal')
fig.update_yaxes(showgrid=False, title_text='Harga')

# Tampilkan plot
fig.show()
